# Notes

- [x] Add progress bar
- [x] Add auto arima
- [ ] Add auto skar
- [ ] Add baselines
- [ ] Add metrics
- [x] Add confidence intervals
- [ ] Add residuals
- [ ] Add simple validation metrics

# Setup

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional, Union, Any
from typing_extensions import Protocol
from copy import deepcopy
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, widgets

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import pmdarima as pm

plt.rcParams["figure.figsize"] = (12, 4)

# Types

In [ ]:
class ScikitModel(Protocol):
    def fit(self, X, y, sample_weight=None): ...
    def predict(self, X): ...
    def score(self, X, y, sample_weight=None): ...
    def set_params(self, **params): ...
    def get_params(self, deep=True): ...

# Data

In [ ]:
stocks = pd.read_csv("../data/all_stocks_5yr.csv", parse_dates=["date"])
stocks

In [ ]:
stocks.info()

In [ ]:
stocks = stocks.set_index("date").groupby("Name").resample("W").mean().reset_index()
stocks

# Metrics

In [ ]:
def error(y, y_fcst):
    err = y - y_fcst
    return err

In [ ]:
def mae(y, y_fcst):
    mae_value = error(y, y_fcst).abs().mean()
    return mae_value

In [ ]:
def mse(y, y_fcst):
    sqe = error(y, y_fcst)**2
    mse_value = sqe.mean()
    return mse_value

In [ ]:
def wape(y, y_fcst):
    err = error(y, y_fcst)
    wape_value = 100 * err.abs().sum() / y.abs().sum() 
    return wape_value

# Models

## Base

In [ ]:
@dataclass
class BaseForecaster:
    key_col: str
    time_col: str
    target_cols: List[str]
    predictor_cols: Optional[List[str]]
    freq: str
    fh: Optional[int] = None
    
    def get_keys(self, data):
        return list(data[self.key_col].unique())
    
    def get_group(self, data, key):
        subdata = data[data[self.key_col] == key].copy()
        return subdata
        
    def get_indexed_series(self, data, key, col):
        subdata = self.get_group(data, key)
        y = subdata.set_index(self.time_col).asfreq(self.freq)[col].copy()
        return y
    
    def get_data_slices(self, data, fh=1, start=0.5, stride=1):
        train_list = []
        data_by_key = data.groupby(self.key_col, group_keys=False)
        n_obs = int(len(data) / len(data_by_key))
        start_cutoff = int(start * n_obs)
        end_cutoff = n_obs - fh
        cutoffs = range(start_cutoff, end_cutoff+1, stride)
        for cutoff in cutoffs:
            data_slice = data_by_key.apply(lambda df: df.iloc[:cutoff, :])
            train_list.append(data_slice)
        return train_list
    
    def get_predictor_slices(self, predictors, fh=1, start=0.5, stride=1):
        predictors = predictors.groupby(self.key_col, group_keys=False).apply(lambda df: df.shift(-fh))
        predictor_slices = []
        for pred_slice in self.get_data_slices(predictors, fh, start, stride):
            predictor_slices.append(pred_slice.groupby(self.key_col).apply(lambda df: df.tail(fh)))
        return predictor_slices
    
    def build_fcst_index(self, data, fh=1):
        if self.fh is not None:
            fh = self.fh
        data_end = data[self.time_col].iloc[-1]
        fcst_start = data_end + pd.Timedelta(1, self.freq)
        fcst_index = pd.date_range(start=fcst_start, periods=fh, freq=self.freq)
        fcst_index.name = self.time_col
        return fcst_index
    
    def af_table(self, y, y_fcst, how="inner"):
        af_df = pd.merge(y, y_fcst, left_index=True, right_index=True, how=how)
        af_df.columns = ["y", "y_fcst"]
        af_df["error"] = af_df["y"] - af_df["y_fcst"]
        return af_df
    
    def af_trim(self, y, y_fcst, how="inner"):
        af_df = self.af_table(y, y_fcst, how=how)
        return af_df["y"], af_df["y_fcst"]
    
    def historical_forecasts(self, data, predictors=None, fh=1, start=0.5, stride=1, progress=True):
        if self.fh is not None:
            fh = self.fh
        hfcst = []
        train_slices = self.get_data_slices(data, fh, start, stride)
        if self.predictor_cols is None or predictors is None:
            predictor_slices = len(train_slices) * [None]
        else:
            predictor_slices = self.get_predictor_slices(predictors, fh, start, stride)
        slices = zip(train_slices, predictor_slices)
        if progress:
            slices = tqdm(list(slices))
        for train, predictor in slices:
            self.fit(train, progress=False)
            fcst = self.predict(fh, predictor, progress=False)
            hfcst.append(fcst)
        return hfcst
    
    def error_table(self, data, predictors=None, fh=1, start=0.5, stride=1, progress=True):
        hfcst = self.historical_forecasts(data, predictors, fh=fh, start=start, stride=stride, progress=progress)
        metrics_dict = dict()
        for key in self.get_keys(data):
            target_metrics = dict()
            for target in self.target_cols:
                metrics_list = list()
                for i, fcst in enumerate(hfcst):
                    y = self.get_indexed_series(data, key, target)
                    y_fcst = self.get_indexed_series(fcst, key, target + "_fcst")
                    y, y_fcst = self.af_trim(y, y_fcst)
                    err = error(y, y_fcst)
                    metrics_list.append(err.reset_index(drop=True))
                metrics_df = pd.concat(metrics_list, axis=1)
                metrics_df.columns = np.arange(len(metrics_df.columns))
                target_metrics[target] = metrics_df.T
            metrics_dict[key] = pd.concat(target_metrics, axis=1)
        scores_df = pd.concat(metrics_dict, axis=1)
        return scores_df
        
    def backtest(self, data, predictors=None, metrics=None, agg=None, fh=1, start=0.5, stride=1, progress=True):
        if metrics is None:
            metrics = [mae]
        hfcst = self.historical_forecasts(data, predictors, fh=fh, start=start, stride=stride, progress=progress)
        metrics_dict = dict()
        for key in self.get_keys(data):
            target_metrics = dict()
            for target in self.target_cols:
                metrics_list = list()
                for i, fcst in enumerate(hfcst):
                    step_metrics = dict()
                    y = self.get_indexed_series(data, key, target)
                    y_fcst = self.get_indexed_series(fcst, key, target + "_fcst")
                    y, y_fcst = self.af_trim(y, y_fcst)
                    for metric in metrics:
                        step_metrics[metric.__name__] = metric(y, y_fcst)
                    metrics_list.append(pd.Series(step_metrics))
                metrics_df = pd.DataFrame(metrics_list)
                target_metrics[target] = metrics_df
            metrics_dict[key] = pd.concat(target_metrics, axis=1)
        scores_df = pd.concat(metrics_dict, axis=1)
        if agg is not None:
            return agg(scores_df)
        return scores_df
    
    def plot_fcst(self, data, fcst, key, target, style="-"):
        target_fcst = target + "_fcst"
        target_lower = target + "_lower"
        target_upper = target + "_upper"
        f, ax = plt.subplots()
        y = self.get_indexed_series(data, key, target)
        y_fcst = self.get_indexed_series(fcst, key, target_fcst)
        y.plot(ax=ax, style=style)
        y_fcst.plot(ax=ax, style=style)
        if target_lower in fcst.columns and target_upper in fcst.columns:
            y_lower = self.get_indexed_series(fcst, key, target_lower)
            y_upper = self.get_indexed_series(fcst, key, target_upper)
            ax.fill_between(x=y_fcst.index, y1=y_lower, y2=y_upper, alpha=0.8, color="lightblue")
        plt.legend()
        
    def plot_forecasts(self, data, fcst):
        interact(
            self.plot_fcst, 
            data=widgets.fixed(data),
            fcst=widgets.fixed(fcst),
            key=self.get_keys(data), 
            target=self.target_cols, 
            style=["-", "."])


## ARIMA

In [ ]:
@dataclass
class ARIMA(BaseForecaster):
    arima_params: dict = field(default_factory=dict)
    fit_params: dict = field(default_factory=dict)
    
    def fit(self, data):
        self.data = data.copy()
        self.model_dict = dict()
        for key in self.get_keys(data):
            model_subdict = dict()
            for target in self.target_cols:
                y = self.get_indexed_series(data, key, target)
                if self.predictor_cols is None:
                    X = None
                else:
                    X = self.get_indexed_series(data, key, self.predictor_cols)
                model = tsa.arima.ARIMA(y, X, **self.arima_params).fit(**self.fit_params)
                model_subdict[target] = model
            self.model_dict[key] = model_subdict
        return self
        
    def predict(self, fh=1, predictors=None):
        fcst_dict = dict()
        for key, model_subdict in self.model_dict.items():
            fcst_subdict = dict()
            for target, model in model_subdict.items():
                if self.predictor_cols is None:
                    X = None
                else:
                    X = self.get_indexed_series(predictors, key, self.predictor_cols)
                y_fcst = model.forecast(fh, exog=X)
                y_fcst.index.name = self.time_col
                y_fcst.name = target + "_fcst"
                y_fcst = y_fcst.reset_index()
                fcst_subdict[target] = y_fcst
            fcst_subdf = pd.concat({key: fcst.set_index("date") for key, fcst in fcst_subdict.items()}, axis=1)
            fcst_subdf.columns = fcst_subdf.columns.droplevel()
            fcst_subdf = fcst_subdf.reset_index()
            fcst_subdf.insert(0, self.key_col, key)
            fcst_dict[key] = fcst_subdf
        fcst = pd.concat(fcst_dict.values())
        return fcst          

## AutoARIMA

In [ ]:
@dataclass
class AutoARIMA(BaseForecaster):
    arima_params: dict = field(default_factory=dict)
    fit_params: dict = field(default_factory=dict)
        
    def fit_for_each_key(self, data, progress):
        model_dict = dict()
        data_keys = self.get_keys(data)
        if progress:
            data_keys = tqdm(data_keys)
        for key in data_keys:
            model_dict[key] = self.fit_for_each_target(data, key)
        return model_dict
            
    def fit_for_each_target(self, data, key):
        model_dict = dict()
        for target in self.target_cols:
            y = self.get_indexed_series(data, key, target)
            if self.predictor_cols is None:
                X = None
            else:
                X = self.get_indexed_series(data, key, self.predictor_cols)
            model = pm.AutoARIMA(**self.arima_params)
            model.fit(y, X, **self.fit_params)
            model_dict[target] = model
        return model_dict
            
    def predict_for_each_key(self, fh, predictors, conf_int, progress):
        fcst_dict = dict()
        model_items = self.model_dict.items()
        if progress:
            model_items = tqdm(model_items)
        for key, models_for_key in model_items:
            fcst_dict[key] = self.predict_for_each_target(key, models_for_key, fh, predictors, conf_int)
        fcst = pd.concat(fcst_dict.values())
        return fcst
        
    def predict_for_each_target(self, key, models_for_key, fh, predictors, conf_int):
        fcst_dict = dict()
        for target, model in models_for_key.items():
            y_fcst_idx = self.build_fcst_index(self.data, fh)
            if self.predictor_cols is None:
                X = None
            else:
                X = self.get_indexed_series(predictors, key, self.predictor_cols)
            if conf_int is None:
                y_fcst = model.predict(n_periods=fh, X=X)
                y_fcst = pd.DataFrame(y_fcst, index=y_fcst_idx, columns=[target + "_fcst"])
            else:
                y_fcst, y_conf = model.predict(n_periods=fh, X=X, return_conf_int=True, alpha=1-conf_int)
                y_fcst = pd.DataFrame(y_fcst, index=y_fcst_idx, columns=[target + "_fcst"])
                y_fcst[target + "_lower"] = y_conf[:, 0]
                y_fcst[target + "_upper"] = y_conf[:, 1]
            y_fcst = y_fcst.reset_index()
            fcst_dict[target] = y_fcst
        fcst_df = pd.concat({target: fcst.set_index(self.time_col) for target, fcst in fcst_dict.items()}, axis=1)
        fcst_df.columns = fcst_df.columns.droplevel()
        fcst_df = fcst_df.reset_index()
        fcst_df.insert(0, self.key_col, key)
        return fcst_df
    
    def fit(self, data, progress=True):
        self.data = data.copy()
        self.model_dict = self.fit_for_each_key(data, progress)
        return self
        
    def predict(self, fh=1, predictors=None, conf_int=None, progress=True):
        fcst = self.predict_for_each_key(fh, predictors, conf_int, progress)
        return fcst    

## SKAR

In [ ]:
@dataclass
class SKAR(BaseForecaster):
    model: ScikitModel = None
    scale_regressors: bool = True
    n_lags: int = 0
        
    def build_lags(self, y):    
        lags = pd.concat([y.shift(i) for i in range(self.n_lags + 1)], axis=1).dropna()
        return lags
    
    def build_target(self, y):
        target = pd.concat([y.shift(-i) for i in range(1, self.fh + 1)], axis=1).dropna()
        return target
    
    def build_fcst_index(self):
        data_end = self.data[self.time_col].iloc[-1]
        fcst_start = data_end + pd.Timedelta(1, self.freq)
        fcst_index = pd.date_range(start=fcst_start, periods=self.fh, freq=self.freq)
        return fcst_index
    
    def fit(self, data):
        if self.model is None:
            self.model = LinearRegression()
        self.data = data.copy()
        self.model_dict = dict()
        self.scaler_dict = dict()
        for key in self.get_keys(data):
            model_subdict = dict()
            scaler_subdict = dict()
            for target in self.target_cols:
                target_data = self.get_indexed_series(data, key, target)
                if self.predictor_cols is not None:
                    predictor_data = self.get_indexed_series(data, key, self.predictor_cols)
                else:
                    predictor_data = None
                y_target = self.build_target(target_data)
                y_lags = self.build_lags(target_data)
                YX = pd.concat([y_target, y_lags, predictor_data], axis=1, join="inner").to_numpy()
                Y = YX[:, :self.fh]
                X = YX[:, self.fh:]
                if self.scale_regressors:
                    scaler = StandardScaler()
                    X = scaler.fit_transform(X)
                    scaler_subdict[target] = scaler
                model = deepcopy(self.model)
                model.fit(X, Y)
                model_subdict[target] = model
            self.model_dict[key] = model_subdict
            self.scaler_dict[key] = scaler_subdict
        return self
    
    def predict(self, fh=None, predictors=None):
        fcst_dict = dict()
        for key, model_subdict in self.model_dict.items():
            fcst_subdict = dict()
            for target, model in model_subdict.items():
                target_data = self.get_indexed_series(self.data, key, [target])
                y_lags = np.flip(target_data.tail(self.n_lags+1).to_numpy())
                if self.predictor_cols is not None:
                    predictor_data = self.get_indexed_series(self.data, key, self.predictor_cols)
                    last_predictors = predictor_data.tail(1).to_numpy()
                    X = np.concatenate([y_lags, last_predictors]).T
                else:
                    X = y_lags.T
                if self.scale_regressors:
                    scaler = self.scaler_dict[key][target]
                    X = scaler.transform(X)
                y_fcst = model.predict(X)
                y_fcst_index = self.build_fcst_index()
                y_fcst = pd.Series(y_fcst.flatten(), index=y_fcst_index)
                y_fcst.index.name = self.time_col
                y_fcst.name = target + "_fcst"
                y_fcst = y_fcst.reset_index()
                fcst_subdict[target] = y_fcst
            fcst_subdf = pd.concat({key: fcst.set_index("date") for key, fcst in fcst_subdict.items()}, axis=1)
            fcst_subdf.columns = fcst_subdf.columns.droplevel()
            fcst_subdf = fcst_subdf.reset_index()
            fcst_subdf.insert(0, self.key_col, key)
            fcst_dict[key] = fcst_subdf
        fcst = pd.concat(fcst_dict.values())
        return fcst
        

# Sandbox

In [ ]:
import darts.models
import darts.metrics
from darts.timeseries import TimeSeries
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
#data = stocks[stocks.Name.isin(["A", "AAL"])]
data = stocks[stocks.Name.isin(["A"])]

In [ ]:
model = AutoARIMA(key_col="Name", time_col="date", target_cols=["open"], predictor_cols=["close"], freq="W")

In [ ]:
hfcst = model.historical_forecasts(data, predictors=data, fh=10)

In [ ]:
model_score = model.backtest(data, predictors=data, metrics=[mae], agg=np.mean, fh=10)

In [ ]:
y = data.set_index("date")[["open"]]
X = data.set_index("date")[["close"]]

In [ ]:
y = TimeSeries(y)
X = TimeSeries(X)

In [ ]:
arima = pm.AutoARIMA()

In [ ]:
arima = darts.models.AutoARIMA()

In [ ]:
score = arima.backtest(y, X, forecast_horizon=10, verbose=True, reduction=None, metric=darts.metrics.mae)

In [ ]:
np.mean(score[1:])

In [ ]:
model.plot_forecasts(data, hfcst[-1])

In [ ]:
y = data[["date", "open"]].set_index("date")

In [ ]:
X = data[["date", "close"]].set_index("date")

In [ ]:
base = BaseForecaster(key_col="Name", time_col="date", target_cols=["open"], predictor_cols=["close"], freq="W")

In [ ]:
model = pm.AutoARIMA()

In [ ]:
model.

In [ ]:
model.fit(y[:-10], X[:-10])

In [ ]:
fcst, conf_int = model.predict(X=X[-10:], return_conf_int=True)

In [ ]:
fcst

In [ ]:
conf_int

In [ ]:
fcst_ix = base.build_fcst_index(data[:-10], fh=10)

In [ ]:
fcst_df = pd.DataFrame(fcst, columns=base.target_cols, index=fcst_ix)

In [ ]:
fcst_df[base.target_cols[0] + "_lower"] = conf_int[:, 0]
fcst_df[base.target_cols[0] + "_upper"] = conf_int[:, 1]

In [ ]:
fcst_df

In [ ]:
fcst = pd.Series(fcst, index=fcst_ix)

In [ ]:
y.plot()
fcst.plot()

In [ ]:
series = TimeSeries(data.set_index("date")[["open"]])
exog = TimeSeries(data.set_index("date")[["close_lag"]])
arima_darts = darts.models.ARIMA(p=12)
arima_darts.fit(series[:-10], exog[:-10])
fcst_darts = arima_darts.predict(n=10, exog=exog[-10:])
scores_darts = arima_darts.backtest(series, exog, forecast_horizon=10, metric=darts.metrics.mae, reduction=None, verbose=True)[1:]
hfcst_darts = arima_darts.historical_forecasts(series, exog, forecast_horizon=10, last_points_only=False, verbose=True)[1:]

print(np.mean(scores_darts))
series.plot()
fcst_darts.plot()

In [ ]:
arima = ARIMA(key_col="Name", time_col="date", target_cols=["open"], predictor_cols=["close_lag"], freq="W", fh=10, arima_params={"order": (12, 1, 0)})
arima.fit(data[:-10])
fcst = arima.predict(fh=10, predictors=data[-10:])
scores = arima.backtest(data, predictors=data, metrics=[mae], fh=10)
hfcst = arima.historical_forecasts(data, predictors=data, fh=10)

print(scores.mean())
arima.plot_forecasts(data, fcst)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
model = MultiOutputRegressor(SVR(kernel="linear"))

In [ ]:
model = MultiOutputRegressor(GradientBoostingRegressor())

In [ ]:
skar = SKAR(key_col="Name", time_col="date", target_cols=["open"], predictor_cols=None, freq="W", fh=10, n_lags=12, model=model, scale_regressors=True)
skar.fit(data[:-10])
fcst = skar.predict()
scores = skar.backtest(data, metrics=[mae])
hfcst = skar.historical_forecasts(data)

print(scores.mean())
skar.plot_forecasts(data, fcst)

In [ ]:
linear_darts = darts.models.LinearRegressionModel(lags=12, lags_exog=12)
linear_darts.fit(series[:-10], exog[:-10])
fcst_darts = linear_darts.predict(n=10, exog=exog[-10:])

series.plot()
fcst_darts.plot()

# Scrapyard